# Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Running ElasticSearch:

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

In [1]:
import json
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
from openai import OpenAI
import tiktoken

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

In [4]:
es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

Look at the `_score` field.

In [5]:
def elastic_search(query, size=5, boost=3, course=None):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": [f"question^{boost}", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }
    if course:
        search_query["query"]["bool"]["filter"] = {}
        search_query["query"]["bool"]["filter"]["term"] = {}
        search_query["query"]["bool"]["filter"]["term"]["course"] = course

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    scores = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        scores.append(hit['_score'])

    return result_docs, scores

query = "How do I execute a command in a running docker container?"
boost = 4
result_docs, scores = elastic_search(query, boost=boost)
print(f"The score for the top ranking result is {round(scores[0], 2)}")

The score for the top ranking result is 84.05


## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

In [6]:
course = "machine-learning-zoomcamp"
size = 3
result_docs, scores = elastic_search(query, size=size, course=course)
print(f"The 3rd question returned by the search engine is: \n{result_docs[2]['question']}")

The 3rd question returned by the search engine is: 
How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

In [7]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

prompt = build_prompt(query, result_docs)
print(f"The length of the resulting prompt is {len(prompt)}")

The length of the resulting prompt is 1462


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query:

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [8]:
encoding = tiktoken.encoding_for_model("gpt-4o")
encode = encoding.encode(prompt)
print(f"Our prompt have {len(encode)} tokens")

Our prompt have 322 tokens


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?

Note: you can replace OpenAI with Ollama. See module 2.

In [9]:
def llm(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

def rag(query):
    search_results, _ = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [10]:
answer = rag(query)
print(f"The response is: \n{answer}")

The response is: 
To execute a command in a running Docker container, follow these steps:

1. First, find the container ID of the running container by using the `docker ps` command. This lists all running containers along with their container IDs.
   ```bash
   docker ps
   ```

2. Identify the container ID of the container in which you want to execute the command.

3. Execute the command inside the container using the `docker exec` command with the `-it` flags to run an interactive terminal. For example, to start a bash shell in the container, you can use:
   ```bash
   docker exec -it <container-id> bash
   ```
Replace `<container-id>` with the actual ID of your running container.

For example:
```bash
docker exec -it abcdef123456 bash
```

This will open an interactive bash shell in the specified running container.


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw1
* It's possible that your answers won't match exactly. If it's the case, select the closest one.